# UUIDs

Using UUID data types in DataJoint tables.


## Background 
Universally Unique Identifiers (UUIDs) provide convenient mechanisms for identifying pieces of information (objects) inside an information system. Various conventions exist. However, general patterns have been established and formalized as RFC 4122.

Comprised of hex digits, UUIDs have the pattern `8-4-4-4-12`, e.g. `e45ba2cc-39db-11e9-8e62-7470fdf23ef1`.

It adds up to 36 characters (32 hex + 4 hyphens), or 16 bytes of information (128 bits).

Please review the documentation of the `uuid` module in the [Python standard library](https://docs.python.org/3/library/uuid.html) to understand how to generate and use UUID objects.

In [1]:
# Author and date
import datetime, getpass
print(getpass.getuser(), datetime.datetime.today())

import datajoint as dj
print('DataJoint Version', dj.__version__)

dimitri 2019-11-03 18:21:46.708257
DataJoint Version 0.12.1


In [2]:
import uuid

In [3]:
# create a schema for this demo
schema = dj.schema('test_uuid')

Connecting dimitri@localhost:3306


Let's create table `Message` that stores messages with a surragate primary key generated as random UUIDs

In [4]:
@schema
class Message(dj.Manual):
    definition = """
    message_id : uuid  # internal message id
    ---
    message_body : varchar(1000) 
    message_timestamp = CURRENT_TIMESTAMP : timestamp 
    """

Let's insert some messages. Note that the UUID field must be inserted as UUID object.

In [5]:
Message.insert1(dict(message_id=uuid.uuid4(), message_body='Hello, world!'))
Message.insert1(dict(message_id=uuid.uuid4(), message_body='Cogito ergo sum'))

In [6]:
Message()

message_id internal message id,message_body,message_timestamp
a36a85a2-508c-4f19-969e-a3c67e18b67e,Cogito ergo sum,2019-11-04 00:21:50
c51f907c-fe20-4d3e-820c-05d32d28f6df,"Hello, world!",2019-11-04 00:21:50


In [7]:
# insert two more
Message.insert((
    dict(message_id=uuid.uuid4(), message_body='I will be back!'),
    dict(message_id=uuid.uuid4(), message_body='Must destroy humans')))

In [8]:
Message()

message_id internal message id,message_body,message_timestamp
7aff5434-17dd-4520-9d2e-f2515bb2cea8,Must destroy humans,2019-11-04 00:21:52
823311ae-142d-4a9f-9b4e-20dc5c49ed43,I will be back!,2019-11-04 00:21:52
a36a85a2-508c-4f19-969e-a3c67e18b67e,Cogito ergo sum,2019-11-04 00:21:50
c51f907c-fe20-4d3e-820c-05d32d28f6df,"Hello, world!",2019-11-04 00:21:50


You may also insert the UUID field as a string as long as it complies with the UUID format.

In [9]:
string_uuid = 'b5bf9baf-b4a5-48ca-9197-ca333e37abaa'
Message.insert1(dict(message_id=string_uuid, message_body='Help!'))

When fetching, the results are returned as `uuid.UUID` objects:

In [10]:
Message.fetch()

array([(UUID('7aff5434-17dd-4520-9d2e-f2515bb2cea8'), 'Must destroy humans', datetime.datetime(2019, 11, 4, 0, 21, 52)),
       (UUID('823311ae-142d-4a9f-9b4e-20dc5c49ed43'), 'I will be back!', datetime.datetime(2019, 11, 4, 0, 21, 52)),
       (UUID('a36a85a2-508c-4f19-969e-a3c67e18b67e'), 'Cogito ergo sum', datetime.datetime(2019, 11, 4, 0, 21, 50)),
       (UUID('b5bf9baf-b4a5-48ca-9197-ca333e37abaa'), 'Help!', datetime.datetime(2019, 11, 4, 0, 21, 54)),
       (UUID('c51f907c-fe20-4d3e-820c-05d32d28f6df'), 'Hello, world!', datetime.datetime(2019, 11, 4, 0, 21, 50))],
      dtype=[('message_id', 'O'), ('message_body', 'O'), ('message_timestamp', 'O')])

and the UUID values can be used in restrictions:

In [11]:
keys = Message.fetch('KEY')
keys[0]

{'message_id': UUID('7aff5434-17dd-4520-9d2e-f2515bb2cea8')}

In [12]:
(Message & keys[0]).fetch1('message_body')

'Must destroy humans'

But strings can also be used as long as it's supplied as a dict-like restriction:

In [13]:
Message & {'message_id': string_uuid}

message_id internal message id,message_body,message_timestamp
b5bf9baf-b4a5-48ca-9197-ca333e37abaa,Help!,2019-11-04 00:21:54


That's all!

In [14]:
# For the curious: This is how the table was declared in SQL
print(schema.connection.query('show create table {}'.format(Message.full_table_name)).fetchone()[1])

CREATE TABLE `message` (
  `message_id` binary(16) NOT NULL COMMENT ':uuid:internal message id',
  `message_body` varchar(1000) NOT NULL,
  `message_timestamp` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP,
  PRIMARY KEY (`message_id`)
) ENGINE=InnoDB DEFAULT CHARSET=latin1


In [15]:
# Clean up after ourselves
schema.drop()

Proceed to delete entire schema `test_uuid`? [yes, No]: yes
